In [39]:
import pandas as pd
import numpy as np
import re
import datetime
import itertools
import json

#reading data
df = pd.read_csv('/Users/codyreece/Desktop/Repos/air-scrape2/air-scrape/Data/Bentonville_Ar.csv')

#removing empty columns
df = df.drop(columns=['host_joined','house_rules', 'specialties_1'])

In [56]:
df['location'].value_counts()

Fayetteville, Arkansas, United States         86
Bella Vista, Arkansas, United States          78
Bentonville, Arkansas, United States          51
Rogers, Arkansas, United States               42
Springdale, Arkansas, United States           23
Centerton, Arkansas, United States            11
Lowell, Arkansas, United States                4
Farmington, Arkansas, United States            2
Township 2, Arkansas, United States            1
Little Flock, Arkansas, United States          1
Washington County, Arkansas, United States     1
Name: location, dtype: int64

In [40]:
#cleaning data
def wrangle(X):

    X['price'] = X['price'].replace(r'\$', '' ,regex=True)
    X['guests'] = X['guests'].str.replace(r'\D', '',regex=True)
    X['rooms'] = X['rooms'].str.replace(r'\D', '',regex=True)
    X['beds'] = X['beds'].str.replace(r'\D', '',regex=True)
    X['baths'] = X['baths'].str.replace(r'\D', '',regex=True)
    X['badge'] = X['badge'].map({'SUPERHOST':1, 'empty':0})
    X = X.rename({'badge': 'superhost'}, axis='columns')
    X['rating'] = X['rating'].replace(regex='empty', value=0)
    X['review_count'] = X['review_count'].replace(regex='empty', value=0)
    X['review_count'] = X['review_count'].replace(r'\D', '',regex=True)

    return X

wrangle(df)

,url,name,header,guests,rooms,beds,baths,facilities,superhost,rating,review_count,price,sp_url,location,amenities
0,/rooms/568774443413437016?adults=4&check_in=20...,Newly Updated ☆ 1 Mile to Coler➱Downtown,Entire residential home in Downtown Bentonville,8,3,4,2,Wifi · Kitchen · Free parking · Self check-in,1,0,0,126,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Hair dryer"", ""Cleaning product..."
1,/rooms/39864150?adults=4&check_in=2022-03-30&c...,"Pintell Trails: Spacious 4 Bed, 3 Bath w/ Garage",Entire residential home in Bentonville,8,4,4,3,Wifi · Kitchen · Free parking · Self check-in,0,0,0,121,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Washe..."
2,/rooms/23161645?adults=4&check_in=2022-03-30&c...,Modern Townhouse at Momentary/8th St Market,Entire townhouse in Downtown Bentonville,8,3,3,25,Wifi · Kitchen · Free parking · Self check-in,1,4.99,101,194,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Shamp..."
3,/rooms/584082363006663396?adults=4&check_in=20...,True North | Back 40 Trail | Sleeps 6,Entire residential home in Bella Vista,6,3,3,2,Wifi · Kitchen · Free parking · Self check-in,1,0,0,122,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bella Vista, Arkansas, United States","{""amenities"": [""Washer"", ""TV"", ""Air conditioni..."
4,/rooms/31653766?adults=4&check_in=2022-03-30&c...,Chic Shack,Entire residential home in Bentonville,6,3,3,2,Wifi · Kitchen · Free parking · Self check-in,1,4.90,31,129,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Hair dryer"", ""Shampoo"", ""Hot w..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,/rooms/14967588?adults=4&check_in=2022-03-30&c...,"4 bedrm, 3 full baths close to Razorback Stadium",Entire residential home in Fayetteville,8,4,4,3,Wifi · Kitchen · Free parking · Self check-in,0,4.58,19,550,https://www.airbnb.com/s/Bentonville--Arkansas...,"Fayetteville, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Clean..."
296,/rooms/52620078?adults=4&check_in=2022-03-30&c...,3 Bedroom home with beautiful back deck-lake view,Entire residential home in Springdale,6,3,25,,Wifi · Kitchen · Free parking · Self check-in,0,0,0,385,https://www.airbnb.com/s/Bentonville--Arkansas...,"Springdale, Arkansas, United States","{""amenities"": [""Lake view"", ""Washer"", ""Iron"", ..."
297,/rooms/12127971?adults=4&check_in=2022-03-30&c...,Beaver Lake Bed & Breakfast,Entire guest suite in Springdale,4,2,2,1,Wifi · Free parking,1,5.0,73,166,https://www.airbnb.com/s/Bentonville--Arkansas...,"Springdale, Arkansas, United States","{""amenities"": [""Hair dryer"", ""Shampoo"", ""Hot w..."
298,/rooms/51329961?adults=4&check_in=2022-03-30&c...,Serene 1 bedroom 1 bath cabin at Beaver Lake!,Entire cabin in Springdale,4,1,1,1,Wifi · Kitchen · Free parking · Self check-in,1,4.91,11,175,https://www.airbnb.com/s/Bentonville--Arkansas...,"Springdale, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Clean..."


In [41]:
LIST_FACILITIES = [
    'Kitchen',
    'Wifi',
    'Free parking',
    'Self check-in'
]


In [42]:
def make_facilities(df, list_facilities=LIST_FACILITIES):
    data_facilities = df.loc[:, ['url', 'facilities']]

    for _top_facility in list_facilities:
        data_facilities[f"facility_{_top_facility}"] = data_facilities['facilities'].fillna('empty').str.split(' · ').apply(lambda x: 1 if _top_facility in x else 0)
    data_facilities.drop('facilities', axis=1, inplace=True)

    return data_facilities

In [43]:
make_facilities(df)

,url,facility_Kitchen,facility_Wifi,facility_Free parking,facility_Self check-in
0,/rooms/568774443413437016?adults=4&check_in=20...,1,1,1,1
1,/rooms/39864150?adults=4&check_in=2022-03-30&c...,1,1,1,1
2,/rooms/23161645?adults=4&check_in=2022-03-30&c...,1,1,1,1
3,/rooms/584082363006663396?adults=4&check_in=20...,1,1,1,1
4,/rooms/31653766?adults=4&check_in=2022-03-30&c...,1,1,1,1
...,...,...,...,...,...
295,/rooms/14967588?adults=4&check_in=2022-03-30&c...,1,1,1,1
296,/rooms/52620078?adults=4&check_in=2022-03-30&c...,1,1,1,1
297,/rooms/12127971?adults=4&check_in=2022-03-30&c...,0,1,1,0
298,/rooms/51329961?adults=4&check_in=2022-03-30&c...,1,1,1,1


In [55]:
amenities_list = list(df.amenities)
amenities_list_string = " ".join(amenities_list)
amenities_list_string = amenities_list_string.replace('{', '')
amenities_list_string = amenities_list_string.replace('}', ',')
amenities_list_string = amenities_list_string.replace('"', '')
amenities_set = [x.strip() for x in amenities_list_string.split(',')]
amenities_set = set(amenities_set)
amenities_set

{'',
 '120\\ HDTV with Amazon Prime Video',
 '2 refrigerators available and also a dorm fridge at the boat dock. Large cooler is also available to take to the dock for the day refrigerator',
 '2-5 years old',
 '40\\ HDTV with Netflix',
 '42\\ HDTV',
 '43\\ HDTV with Roku',
 '47\\ HDTV with Roku',
 '5-10 years old',
 '50\\ HDTV',
 '50\\ HDTV with Chromecast',
 '50\\ TV with Chromecast',
 '55\\ HDTV with Amazon Prime Video',
 '55\\ HDTV with Disney+',
 '55\\ HDTV with Netflix',
 '55\\ HDTV with Roku',
 '55\\ TV with Apple TV',
 '60\\ HDTV with Netflix',
 '65\\ HDTV with Roku',
 '65\\ TV',
 '65\\ TV with Netflix',
 '70\\ HDTV with Hulu',
 '70\\ HDTV with standard cable',
 '72\\ HDTV',
 '75\\ TV with Amazon Prime Video',
 'Air conditioning',
 'All new appliances refrigerator',
 'All new appliances stove',
 'Amazon Prime Video',
 'Apple TV',
 'Assorted body soap',
 'BBQ grill',
 'BBQ grill: charcoal',
 'BBQ grill: gas',
 'BBQ grill: wood-burning',
 'Baby bath',
 'Baby monitor',
 'Baby safet